<a href="https://colab.research.google.com/github/PorkPy/LSTM-Force-Predictor/blob/master/predictor_17_8_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
% reset -f
from __future__ import print_function
from __future__ import division
import torch
import torch.nn.functional as F
from torch import nn, optim
import sys
import os
import glob
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.ticker as plticker
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
import random
import time
from datetime import datetime
from subprocess import call
import warnings
warnings.filterwarnings("ignore")


In [25]:
'''              ### MODEL HYPER-PARAMETER SETTINGS ###
-------------------------------------------------------------------------------
Here, the model hyper-parameters can be set, along with the save model path 
and the warm start model parameters path if using a pretrained model. 
'''
model_num  = '1001'        ## Unique model number for saving new model.
model_dir  = 'model1001'   ## New directory name to save new model to. 
params     = '1000_v92'     ## Pretrained model params to load...
warm_start = True        ## ...and if to load them or not.
seq_length = 1000          ## The length of the trajectory slice trained on.
epochs     = 101          ## Number of full passes through the whole dataset.
hidden     = 64           ## Number of nodes in the LSTM layers.
lstm_layers = 2
lr         = 0.0005       ## Learning rate.
feature_num   =  6        ## 4 features for joint data, 6 features for cartesian data.
fc         = 1           ## Number of fully connected layers. 1 or 2.
dropout = 0.5
random_seed = 42           ## Used to seed the random number generator for reproducibility.

## Remarks about this particular test.
notes = ("""  
            
""")

np.random.seed(random_seed)
torch.manual_seed(42) ## We should keep the same torch seed for the same weight initialisation.
path       = f"/content/drive/My Drive/PhD/PhD/lstm/{model_dir}/" ## Save directory.

#------------------------------------------------------------------------------
## I increased the batch size and lr by 1 order.

## Functions to fetch hyper-parameters
def model_number():
    return model_num

def load_params():
    return params

def model_directory():
    return model_dir

def get_seq_length():
    return seq_length

def get_epochs():
    return epochs

def get_warm_start():
    return warm_start

def get_hidden():
    return hidden

def get_lr():
    return lr

def get_path():
    return path

def get_features():
    return feature_num

def get_fc():
    return fc

def get_lstm_layers():
    return lstm_layers

def get_random_seed():
    return random_seed

def get_dropout():
    return dropout

## Dictionary with which to save paramers.
param = {'Model Num':model_num, 
          'Seq Length': seq_length,
          'Epochs': epochs,
          'Warm Start': (warm_start),
          'Pretrained on': params,
          'Hidden Size': hidden,
          'Learning Rate': lr,
          'features': feature_num, 
          'Num LSTM Layers':lstm_layers,
          'Num FC Layers':fc,
          'Dropout': dropout,
          'Random Seed': random_seed,
          'Data/Notes': notes
}

## Create new directory in perent directory to save parameters.
try:
    os.makedirs(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)


## create a pandas data frame of the model parameters and save to csv.
param = pd.DataFrame(param, index=[0])
param.to_csv(path + "lstm_params.csv", index=False)


Successfully created the directory /content/drive/My Drive/PhD/PhD/lstm/model1001/ 


In [26]:
class ForcePredictor(nn.Module):

    def __init__(self, n_features, n_hidden, seq_len, n_layers=2, ignore_zero=True):
        super(ForcePredictor, self).__init__()
        dropout = get_dropout()

        if torch.cuda.is_available():
            device = torch.device("cuda:0")
            print("Running on the GPU")
        else:
            device = torch.device("cpu")
            print("Running on CPU")

        self.n_hidden = n_hidden
        self.seq_len = seq_len
        self.n_layers = n_layers

        self.lstm = nn.LSTM(
          input_size=n_features,
          hidden_size=n_hidden,
          num_layers=n_layers)#,
          #dropout= dropout)
        
        
        fc = get_fc() ## get num of FC layers


        if fc == 1:
            self.linear1 = nn.Linear(in_features=n_hidden, out_features=3)
            
        elif fc == 2:
            self.linear1 = nn.Linear(in_features=n_hidden, out_features=60)
            self.linear2 = nn.Linear(in_features=60, out_features=3)

        elif fc == 3:
            self.linear1 = nn.Linear(in_features=n_hidden, out_features=60)
            self.linear2 = nn.Linear(in_features=60, out_features=60)
            self.linear3 = nn.Linear(in_features=60, out_features=3)
        
    def reset_hidden_state(self):
        self.hidden = (
            torch.zeros(self.n_layers, self.seq_len, self.n_hidden).to(device),
            torch.zeros(self.n_layers, self.seq_len, self.n_hidden).to(device)
        )

    ## Forward Function.
    
    def forward(self, sequences):

        '''                    ## Forward Method ##
        -----------------------------------------------------------------------
        This Method takes the input and passes it through each of the network 
        layers.
        The number of fully connected layers the input gets passed through is
        dependent on the number stipulated in the network hyper-parameters at
        the beginning of the notebook.
        ----------------------------------------------------------------------- 
        '''
        
        fc = get_fc() ## get num of FC layers

        lstm_out, self.hidden = self.lstm(sequences.view(len(sequences), self.seq_len, -1),self.hidden)
        last_time_step = lstm_out.view(self.seq_len, len(sequences), self.n_hidden)[-1]

        if fc == 1:
            y_pred = self.linear1(last_time_step)

        if fc == 2:
            y_pred = F.leaky_relu(self.linear1(last_time_step))
            y_pred = self.linear2(y_pred)

        if fc == 3:
            y_pred = F.leaky_relu(self.linear1(last_time_step))
            y_pred = F.leaky_relu(self.linear2(y_pred))
            y_pred = self.linear3(y_pred)

       

        return y_pred

In [27]:
model = ForcePredictor(
      n_features=get_features(), 
      n_hidden= get_hidden(), #32, #64
      seq_len=seq_length, 
      n_layers=lstm_layers
    )


lr = get_lr()
loss_fn = torch.nn.MSELoss(reduction='mean')
device = torch.device("cuda:0")
loss_fn = loss_fn.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)#0.0007 
print("learning rate =", lr) 
num_epochs = get_epochs() #1600 #600
path = get_path()
seq_length = get_seq_length()

#--------------------------------------------------------------------------
'''                   ## Get Model Parameters ##
---------------------------------------------------------------------------
If you want to restart training from an earlier model, first - it should be
stipulated in the hyper-parameter setting at the beginning of the notebook
by setting 'warm_start' = true, and adding the path to the saved model 
location.


'''
start_epoch = 0
warm_start = get_warm_start()
if warm_start == True:      
    params = load_params() # model num and version num: 4_v100.
    PATH = f"/content/drive/My Drive/PhD/PhD/lstm/model_params{params}.pt"     
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    for state in optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()
    start_epoch = checkpoint['epoch']
    loss_fn = checkpoint['loss']
    loss_fn = loss_fn.to(device)

model = model.to(device)


def lighten_color(color, amount=0.5):
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2])

def tests(model_name):
    
    ## fetch parameters
    seq_len = get_seq_length()
    model_name = model_name
    model_dir = model_directory()
    path = get_path()
    features_num = get_features()

    
    ## Create new directory in perent directory
    path = path + f"{model_name}/"
    try:
        os.makedirs(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)

    stats_list = []
    pdf = PdfPages(path + f"testing_traj_pics_{model_name}.pdf")
    fig = plt.figure()

    
    #model = model.to(device)
    #print(model)
    #print("testing weights", model.linear1.weight.data) # Check weights are being updated.

    #model.reset_hidden_state()
    for traj in range(len(test_batches)):
        model.reset_hidden_state()
        whole_traj = []
        whole_true = []

        '''Although it is possible to feed in the entire trajectory and get
        the same prediction results, this cannot be done within the same model 
        instance, as the sequence length is fixed. 
        '''
        for start_seq in range(int(1000/seq_length)): 
            start_seqx = start_seq*seq_length ## get the next sequence start position
            #model.reset_hidden_state()

            Xtest, ytest = get_test_batch(traj, start_seqx)
            model.eval()
            with torch.no_grad():
                
                x = iter(Xtest)
                test_seq = Xtest[0].reshape(-1,seq_len,features_num)#.reshape(1,200,4) # input first sequence from trajectory/batch
                preds = [] # create a list to store predictions.
                for i in range(len(Xtest)): # for each sequence i in the trajectory,
                    y_test_pred = model(test_seq).to(device)# send sequence to model,
                    pred = torch.flatten(y_test_pred).cpu() # reshape the model output,
                    preds.append(np.asarray(pred)) # and append to the list of predictions - preds.
                    new_seq = next(x).reshape(-1,seq_len,features_num)#.reshape(1,200,4) # Change sequence to the next one in the list.
                    test_seq = torch.cuda.FloatTensor(new_seq).view(1, seq_len, -1) # change sequence to a torch Tensor
            whole_traj.append(preds)
            whole_true.append(ytest)
        whole_true = np.array(whole_true).reshape(-1,)
        ## rescale the output predictions
        preds = target_scaler.inverse_transform(whole_traj).reshape(-1,3)
        ## Vector summation - the vector sum of the 3 output predictions
        force_vec = np.sqrt((preds[:,0]**2)+(preds[:,1]**2)+(preds[:,2]**2))
        
        ytest = whole_true
        preds = force_vec ## reset name to comply with existing code.
        #display(force_vec)
        
        #Mean Absolute Error
        MAE_list = []
        for i,j in zip(preds, ytest):
            error = np.abs(i-j)
            MAE_list.append(error)
        MAE = float("{:.3f}".format(np.mean(MAE_list)))
        #print("MAE","{:.3f}".format(MAE),'N')

        # Coefficient of Variance
        mean = np.mean(data.iloc[:,-1]) # mean of all dependent variables.
        cov_list = []
        for i,j in zip(preds, ytest):
            sq_dev = (i-j)**2
            cov_list.append(sq_dev)    
        MSD = np.mean(cov_list) # mean square deviation
        RMSD = np.sqrt(MSD) # root mean square deviation
        cov = RMSD/mean # coefficient of variance
        RMSD = float("{:.3f}".format(RMSD))
        cov =  float("{:.3f}".format(cov))
        #print("COV:","{:.3f}".format(cov))
        
    
        my_dict = {'Trajectory':traj,
                'MAE': MAE, 
                'RMSD':RMSD,
                'cov': cov, # Used to normalise the RMSD accross all the data
        }
        stats_list.append(my_dict)

        # Plot forces
        predicted_cases = preds
        true_cases = ytest
        # Add title and axis names
        plt.title(f'Force Trajectory {traj}');
        plt.xlabel('Sample num');
        plt.ylabel('Force (N)');
        plt.tight_layout();
        plt.grid(True)
        plt.ylim(-1, 70)
        plt.plot(true_cases, color=lighten_color('b', 1.7), linewidth=3.0, label='Real Force');
        x = [i for i in range(1000)]
        #plt.scatter(x, predicted_cases, marker='.', s=1, c='r', label='Predicted Force');
        plt.legend(loc=2, prop={'size': 6})

        from numpy.polynomial import Polynomial
        x = [i for i in range(1000)]
        y = predicted_cases
        
        p = Polynomial.fit(x, y, 50)
        plt.plot(*p.linspace(), color=lighten_color('r', 1.0), linewidth=1.0, label='Real Force');


        # save the current figure
        pdf.savefig(fig);
        # destroy the current figure
        plt.clf()

    pdf.close()
    stats_list = pd.DataFrame(stats_list)
    return stats_list

###############################################################################

def stats(stats_list2, model_name):
    
 
    ## Get the mean of MAE, RMSD and cov.
    mean_list = {
                'MAE' :float("{:.3f}".format(np.mean(stats_list2['MAE']))),
                'RMSD':float("{:.3f}".format(np.mean(stats_list2['RMSD']))),
                'cov' :float("{:.3f}".format(np.mean(stats_list2['cov'])))
    }
    ## Get the std-dev of MAE, RMSD and cov.
    std_dev = {
                'MAE' :float("{:.3f}".format(np.std(stats_list2['MAE']))),
                'RMSD':float("{:.3f}".format(np.std(stats_list2['RMSD']))),
                'cov' :float("{:.3f}".format(np.std(stats_list2['cov'])))
    }
    ## Get the max value of MAE, RMSD and cov.
    max_list = {
                'MAE' :float(stats_list2['MAE'].max()),
                'RMSD':float(stats_list2['RMSD'].max()),
                'cov' :float(stats_list2['cov'].max())
    }
    ## append above dicts to stats_list2.
    stats_list2 = stats_list2.append(mean_list, ignore_index=True).fillna('Grand Mean')
    stats_list2 = stats_list2.append(std_dev, ignore_index=True).fillna('Standard Dev')
    stats_list2 = stats_list2.append(max_list, ignore_index=True).fillna('Max Value')

    #display(stats_list2)
    path = get_path()
    
    ## Create new directory in perent directory
    path = path + f"{model_name}/"
    model_name = model_name

    ## save stats_list as .csv in same directory as trajectory plots.
    stats_list2.to_csv(path + f"lstm_model_metrics_{model_name}.csv", index=False)
    return stats_list2

###############################################################################

## Get a Gaussian Distribution of the MAE, RMSD and cov.
def gauss_plot(stats_list2, name, error_type, num):
    model_name = name
    model_dir = model_directory()
    path = get_path()
    path = path + f"{model_name}/"

    error = error_type ## Either; MAE, RMSD or cov.
    pdf = PdfPages(path + f"gauss_pic_{error}.pdf")
    fig = plt.figure()
    
    # define constants
    mu = np.mean(stats_list2.iloc[:-3,num]) 
    sigma = np.sqrt(np.var(stats_list2.iloc[:-3,num]))
    x1 = np.min(stats_list2.iloc[:-3,num])
    x2 = np.max(stats_list2.iloc[:-3,num])
    

    # calculate the z-transform
    z1 = ( x1 - mu ) / sigma
    z2 = ( x2 - mu ) / sigma

    x = np.arange(z1, z2, 0.001) # range of x in spec
    x_all = np.arange(-10, 10, 0.001) # entire range of x, both in and out of spec
    # mean = 0, stddev = 1, since Z-transform was calculated
    y = norm.pdf(x,0,1);
    y2 = norm.pdf(x_all,0,1);

    # build the plot
    fig, ax = plt.subplots(figsize=(9,6));
    #plt.style.use('fivethirtyeight');
    ax.plot(x_all,y2);

    ax.fill_between(x,y,0, alpha=0.3, color='b');
    ax.fill_between(x_all,y2,0, alpha=0.1);
    ax.set_xlim([-4,4]);
    ax.set_xlabel('# of Standard Deviations Outside the Mean');
    ax.set_yticklabels([]);
    ax.set_title(f'{model_name} {error} Std Dev');

    plt.savefig('normal_curve.png', dpi=72, bbox_inches='tight');
    plt.grid(True);
    plt.tight_layout();
    #plt.show()
    # save the current figure
    pdf.savefig(fig);
    ## destroy the current figure
    plt.clf()

    # close the object
    pdf.close()

###############################################################################

## Get a PDF of the MAE, RMSD and cov.
def prob_dist(stats_list2, name, error_type, num):    
    model_name = name
    model_dir = model_directory()
    path = get_path()
    path = path + f"{model_name}/"


    error = error_type
    pdf = PdfPages(path + f"prob_dist_pic_{error}.pdf")
    fig = plt.figure()

    import seaborn as sns
    sns.distplot(stats_list2.iloc[:-3,num], color="darkslategrey");
    plt.xlabel("Force [newtons]", labelpad=14);
    plt.ylabel("Probability of Occurence", labelpad=14);
    plt.title(f"Probability Distribution of {error}", fontsize=20);
    plt.grid(True);
    plt.tight_layout();

    #plt.show()
    # save the current figure
    pdf.savefig(fig);
    # destroy the current figure
    plt.clf()
    plt.close('all') ## added this due to runtime warning, more than 20 figs open
    # close the object
    pdf.close()


Running on the GPU
learning rate = 0.0005


In [28]:
def test_runner(name):   
    stats_df = tests(name) # Run tests on testing data and save generated plots to Google Drive
    stats(stats_df, name) # Record stats and save to Google Drive
    for i in range(1,4): # 1 to 3 = the colunms in the stats_list DataFrame
        if i ==1:
            error_type = 'MAE' # mean absolur error
        elif i == 2:
            error_type = 'RMSE' # root mean squared error
        elif i == 3:
            error_type = 'cov' # coefficient of variance

        prob_dist(stats_df, name, error_type, i) # Gen prob_dist and save to GD
        
        gauss_plot(stats_df, name, error_type, i) # Gen Gauss plots and save to GD
    print("Done")

In [29]:
## Get num
feature_num = get_features()
url = 'https://raw.githubusercontent.com/PorkPy/LSTM-Force-Predictor/master/80k_data/cart_data_plus_rotation.csv'

url2 = 'https://raw.githubusercontent.com/PorkPy/LSTM-Force-Predictor/master/80k_data/4_joints_3_force_1_forceVec.csv'

url3 = 'https://raw.githubusercontent.com/PorkPy/LSTM-Force-Predictor/master/80k_data/mean_force_data.csv'


data = pd.read_csv(url)
data2 = pd.read_csv(url2)
data3 = pd.read_csv(url3)
main_seq = data

In [30]:
features = data
feature_scaler = StandardScaler()
features = feature_scaler.fit_transform(features)

targets = data2.iloc[:,:3]
target_scaler = StandardScaler()
targets = target_scaler.fit_transform(targets)

force_vec = pd.DataFrame(data3.iloc[:,-1])

features = pd.DataFrame(features)
targets = pd.DataFrame(targets)
data = pd.concat([targets, features, force_vec], axis=1)
data.columns = [['Fx', 'Fy', 'Fz', 'x', 'y', 'z', 'Rx', 'Ry', 'Rz', 'force vec']]
display(data)

,Fx,Fy,Fz,x,y,z,Rx,Ry,Rz,force vec
0,0.229527,0.132190,0.064890,-1.136618,0.408166,-0.550866,0.521896,-0.377128,-0.302726,0.000000
1,0.218995,0.116413,0.061217,-1.136679,0.408226,-0.551257,0.521890,-0.376738,-0.302744,0.000000
2,0.229527,0.125879,0.063053,-1.136613,0.408126,-0.550886,0.521903,-0.375445,-0.302700,0.000000
3,0.232159,0.124827,0.066726,-1.136642,0.408244,-0.551756,0.521884,-0.375788,-0.302735,0.000000
4,0.224260,0.140606,0.063053,-1.136663,0.408174,-0.550305,0.521909,-0.376406,-0.302726,0.000000
...,...,...,...,...,...,...,...,...,...,...
71995,0.303249,0.285764,0.239361,1.972838,0.565879,2.281913,-1.911302,-1.151414,0.913013,5.038129
71996,0.342744,0.335202,0.239361,1.929224,0.529467,2.223429,-1.911355,-1.136161,0.905846,4.659258
71997,0.292717,0.319424,0.220995,1.886598,0.497976,2.169416,-1.911478,-1.115511,0.900044,4.305321
71998,0.463860,0.241586,0.362408,1.845012,0.474289,2.109868,-1.911668,-1.100489,0.895936,3.918005


In [31]:
n=1000  ## num samples per trajectory/sequence.
batchesx = [data[i:i + n] for i in range(0, len(data), n)] ## a list comprehension to build the data batches.
print(len(batchesx))

random.seed(get_random_seed())
random.shuffle(batchesx)


batches = batchesx[:60] ## Training batches up to the 60th sequence/trajectory.
val_batches = batchesx[60:] ## Validation batches starting from the 60th sequence.

## Append extra validation batches to even the number training and validation batches.
## This is because the training loop performs a validation test on each iteration
## and so always needs something to validate against.  
while len(val_batches) < len(batches): 
    for i in batchesx[60:]:
        val_batches.append(i)
random.shuffle(val_batches)

test_batches = batchesx[60:] ## Testing batches, same as validation batches, without the appendages.
print(len(batches), len(val_batches), len(test_batches))

72
60 60 12


In [32]:
def get_test_batch(batch_number, start_seq):
    
    seq_size = get_seq_length() ## 1000 for testing 50-100 for training
    features_num = get_features()

    X_test = []

    data = test_batches[batch_number].reset_index(drop=True)
    data= data[start_seq:seq_size+start_seq]

    if features_num == 4:
        features = data[['joint_0', 'joint_2', 'joint_4', 'joint_5']]
    else:
        features = data[['x', 'y', 'z', 'Rx', 'Ry', 'Rz']]
    features = np.asarray(features)
    y_test = data.iloc[:,-1]
    
    for i in range(len(features)):           
   
        X =(features[:i+1])
        an_array = np.array(X)
        shape = np.shape(X)
        temp = np.zeros((seq_size, features_num))
        temp[(seq_size-shape[0]):,:shape[1]] = an_array
        X_test.append(temp)

    
    X_test = torch.cuda.FloatTensor(X_test)
    return(X_test, y_test)
  

In [33]:
name = f'model1001_v92'
test_runner(name)

Successfully created the directory /content/drive/My Drive/PhD/PhD/lstm/model1001/model1001_v92/ 
Done


<Figure size 432x288 with 0 Axes>

<Figure size 648x432 with 0 Axes>